In [1]:
# Imports
from sys import argv
from os import path
from enum import Enum
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# !pip install crypten

In [3]:
import crypten
import crypten.communicator as comm
crypten.init()
torch.set_num_threads(1)
import crypten.mpc as mpc
import crypten.communicator as comm

In [4]:
# Constants

NUMEXEC = 10
NUMFOLD = 10
SEEDFACTOR = 10
DATASET = '../dataset/'
RESULT = '../result/'
FRAMES = [4,5,6,7,8]
DATASETTYPE = 'va'
RESULTFILE = 'results.csv'
LEARNINGRATE = 0.01
BATCHSIZE = 10
HIDDENSIZE = 10
TESTFRAC = 0.1

PARAM = "-L 0.3 -M 0.2 -N 500 -V 0 -S %%SEED%% -E 20 -H %%HIDDEN_LAYERS%%"
HIDDENLAYER = ["a", "40", "30", "20", "10"]

class CoordinateEnum(Enum):
    X = 'X'
    Y = 'Y'
    Z = 'Z'

class Sensor:
    def __init__(self, name, key, *argv):
        self.name = name
        self.key = key
        self.coordinates = list(argv)   

class SensorEnum(Enum):
	LINEAR_ACCELERATION_EARTH = Sensor("aceleracaoLinearTerra", "AclLinE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)
	ACCELEROMETER_EARTH = Sensor("acelerometroTerra", "AcelE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)
	MAGNETIC_FIELD_EARTH = Sensor("campoMagneticoTerra", "MagE", CoordinateEnum.Y, CoordinateEnum.Z)
	GYROSCOPE_EARTH = Sensor("giroscopioTerra", "GirE", CoordinateEnum.X, CoordinateEnum.Y, CoordinateEnum.Z)

class EventEnum(Enum):
    AGGRESSIVE_LEFT_TURN = "curva_esquerda_agressiva"
    AGGRESSIVE_RIGHT_TURN = "curva_direita_agressiva"
    AGGRESSIVE_LEFT_LANE_CHANGE = "troca_faixa_esquerda_agressiva"
    AGGRESSIVE_RIGHT_LANE_CHANGE = "troca_faixa_direita_agressiva"
    AGGRESSIVE_ACCELERATION = "aceleracao_agressiva"
    AGGRESSIVE_BRAKE = "freada_agressiva"
    NON_AGGRESSIVE = "evento_nao_agressivo"
 
events = {'curva_esquerda_agressiva': 0, 
          'curva_direita_agressiva': 1, 
          'troca_faixa_esquerda_agressiva': 2, 
          'troca_faixa_direita_agressiva': 3, 
          'aceleracao_agressiva': 4, 
          'freada_agressiva': 5, 
          'evento_nao_agressivo': 6
         }

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, inputSize, hiddenSize, numClasses) -> None:
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(inputSize,hiddenSize)
        self.l2 = nn.Linear(hiddenSize,hiddenSize)
        self.l3 = nn.Linear(hiddenSize,hiddenSize)
        self.l4 = nn.Linear(hiddenSize,numClasses)

    def forward(self,x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x
#         return F.log_softmax(x,dim=1)


In [6]:
# class NeuralNet(crypten.nn.Module):
#     def __init__(self, inputSize, hiddenSize, numClasses) -> None:
#         super(NeuralNet, self).__init__()
#         self.l1 = crypten.nn.Linear(inputSize,hiddenSize)
#         self.l2 = crypten.nn.Linear(hiddenSize,hiddenSize)
#         self.l3 = crypten.nn.Linear(hiddenSize,hiddenSize)
#         self.l4 = crypten.nn.Linear(hiddenSize,numClasses)

#     def forward(self,x):
#         x = (self.l1(x)).relu()
#         x = (self.l2(x)).relu()
#         x = (self.l3(x)).relu()
#         x = self.l4(x)
#         return x
# #         return F.log_softmax(x,dim=1)

In [7]:
ALICE = 0
BOB = 1

In [8]:
class RecordHandlerCrypt:
    def __init__(self,values,events):
        self.values = values
        self.events = events
        self.numClasses = len(events)
        self.inputSize = values[0][:-2].size
    
    def segregate(self, testFrac):
        
        np.random.shuffle(self.values)
    
        features = torch.Tensor(
            np.array( [i[:-2].astype(np.float32) for i in self.values])
        )
        
        labels = torch.Tensor(
            np.array( [ self.events[i[-1]] for i in self.values])
        ).type(torch.LongTensor)
        
        label_eye = torch.eye(self.numClasses)
        one_hot = label_eye[labels]
        
        #TODO: Also store quadro0 for future refrences (giving results)
        
        testSize = int(testFrac*len(features))
        
#         ohe_enc = crypten.cryptensor(ohe, src=ALICE, requires_grad=True)
#         features_enc = crypten.cryptensor(features, src=ALICE, requires_grad=True)
#         labels_enc = crypten.cryptensor(labels, src=ALICE, requires_grad=True) 
        self.features = features
        self.labels = labels
        self.one_hot = one_hot
        
        self.testFeatures = features[:testSize]
        self.testLabels = labels[:testSize]
        self.testOnehot = one_hot[:testSize]
        
        self.trainFeatures = features[testSize:]
        self.trainLabels = labels[testSize:]
        self.trainOnehot = one_hot[testSize:]
               

In [9]:
def train():
    # train
    for epoch in range(NUMEXEC):
        batch_loss = 0
        for i in tqdm(range(0, len(handler.trainFeatures), BATCHSIZE)): 

            features = handler.trainFeatures[i:i+BATCHSIZE].view(-1,handler.inputSize)
            labels = handler.trainLabels[i:i+BATCHSIZE]

            net.zero_grad()

            # forward
            output = net(features)
#             loss = F.nll_loss(outputs,labels)
            loss = nn.CrossEntropyLoss()
            loss = loss(output,labels)

            # backward
            loss.backward()    
            optimizer.step()
            batch_loss = batch_loss + loss
        print(batch_loss)



In [10]:
def predictTest():
    # test
    with torch.no_grad():
        correct = 0
        samples = 0

        for i in range(len(handler.testFeatures)):
            features = handler.testFeatures[i].view(-1,handler.inputSize)
            label = handler.testLabels[i]

            output = net(features)
            if(torch.argmax(output) == label):
                correct += 1
            samples += 1 

        acc = 100.0 * correct /samples
        print(f'accuracy = {acc}')


In [11]:
def predictTrain():
    # test
    with torch.no_grad():
        correct = 0
        samples = 0

        for i in range(len(handler.trainFeatures)):
            features = handler.trainFeatures[i].view(-1,handler.inputSize)
            label = handler.trainLabels[i]

            output = net(features)
            if(torch.argmax(output) == label):
                correct += 1
            samples += 1 

        acc = 100.0 * correct /samples
        print(f'accuracy = {acc}')

In [12]:
path = f'../datasets/va_acelerometroTerra_nq8.csv'
file = pd.read_csv(path)
values = file.values
handler = RecordHandlerCrypt(values,events)
handler.segregate(TESTFRAC)
net = NeuralNet(handler.inputSize, HIDDENSIZE, handler.numClasses)
optimizer = optim.Adam(net.parameters(),lr = LEARNINGRATE) 

In [13]:
# train()
# predictTrain()
# predictTest()

In [14]:
# path = f'../datasets/va_acelerometroTerra_nq8.csv'
# file = pd.read_csv(path)
# values = file.values
# handler = RecordHandlerCrypt(values,events)
# handler.segregate(TESTFRAC)
# net = NeuralNet(handler.inputSize, HIDDENSIZE, handler.numClasses)
dummy_input = torch.empty((1, handler.inputSize))
model = crypten.nn.from_pytorch(net, dummy_input)
# loss_enc = crypten.nn.loss.CrossEntropyLoss()
# loss_enc = crypten.nn.MSELoss()

In [15]:
# train()
# predictTrain()
# predictTest()

In [16]:
@mpc.run_multiprocess(world_size=1)
def run_encrypted_training(model):
    arr = []
    loss_arr = []
    model.train()
    model.encrypt(src=0)
#     loss = crypten.nn.MSELoss()
    loss = crypten.nn.loss.CrossEntropyLoss()
#     optimizer = crypten.optim.SGD(net.parameters(),lr=0.01)
    rank = comm.get().get_rank()
    for epoch in range(100):
        running_loss = 0.0
        for i in tqdm(range(0, len(handler.trainFeatures), BATCHSIZE)): 

#             features = handler.trainFeatures[i:i+BATCHSIZE].view(-1,handler.inputSize)
#             labels = handler.trainLabels[i:i+BATCHSIZE]
#             one_hot = handler.trainOnehot[i:i+BATCHSIZE]
            
            features = handler.features[i:i+BATCHSIZE].view(-1,handler.inputSize)
            labels = handler.labels[i:i+BATCHSIZE]
            one_hot = handler.one_hot[i:i+BATCHSIZE]
            
            inputs_enc = crypten.cryptensor(features)
            labels_enc = crypten.cryptensor(one_hot)

            output = model(inputs_enc)
            out_plain = output.get_plain_text()
            print(torch.argmax(out_plain, dim = -1))
            
            model.zero_grad()
            loss_value = loss(output,labels_enc)
            loss_value.backward()
            model.update_parameters(LEARNINGRATE)
            
            loss_value_decoded = loss_value.get_plain_text().item()
#             loss_arr.append(loss_value_decoded)
            running_loss += loss_value_decoded
        print(running_loss)

run_encrypted_training(model)

  0%|                                                                                            | 0/50 [00:00<?, ?it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


  2%|█▋                                                                                  | 1/50 [00:00<00:06,  7.59it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


  4%|███▎                                                                                | 2/50 [00:00<00:05,  8.33it/s]

...

 98%|█████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [00:06<00:00,  8.28it/s]

tensor([6, 5, 5, 5, 5, 6, 6, 5, 4, 5])


100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  8.02it/s]


52.66764831542969


[None]

In [23]:
def eval_crypt():
    features = handler.testFeatures.view(-1,handler.inputSize)
    label = handler.testLabels
    features = crypten.cryptensor(features, src=ALICE)
    output = model(features)
    output = output.get_plain_text()
    output = torch.argmax(output, dim = -1)
    print(output)
    acc = torch.sum(output == label)
    print(f'accuracy = {acc*100/label.size()[0]}')

eval_crypt()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])
accuracy = 18
